In [ ]:
import numpy as np
import control as con
from numpy import linalg as LA

import cvxpy
import optim_tools as optim_tools#own file with helper

In [ ]:
###########################
# Hydraulischer Aktor     #
###########################

A0 = np.matrix([[0,   1,       0],
                [-10, -1.167, 25],
                [0,   0,    -0.8]])
print "Eigenvalues: {}".format(LA.eigvals(A0))
#a = -A[-1,:].T ### !!!!!
#print a
b0 = np.matrix([[0],[0],[2.4]])
c0 = np.matrix([1, 0, 0])
d0 = np.matrix([0])
u_max = 10.5
n = 3

X0 = [np.matrix([-20.0, -10, -10]).T,
      np.matrix([-20.0, -10, 10]).T,
      np.matrix([-20.0,  10, -10]).T,
      np.matrix([20.0,  -10, 10]).T,
      np.matrix([20.0,  -10, -10]).T,
      np.matrix([20.0,   10, 10]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c


# Convert to Normalform
ss, T = con.canonical_form(con.ss(A0, b0, c0, d0), form='reachable')

A = optim_tools.reverse_x_order(np.matrix(ss.A))
a = -A[-1][:].T #!!!!

b = optim_tools.reverse_x_order(np.matrix(ss.B))
c = optim_tools.reverse_x_order(np.matrix(ss.C))
d = optim_tools.reverse_x_order(np.matrix(ss.D)) # == 0!

print "A:\n", A
print "a:\n", a
print "b:\n", b
print "c:\n", c

In [ ]:
# TODO: Was ist m? Kann man das berechnen oder wird das festgelegt m<=2n-1, meistens n+1?

def get_a_list(a, Q, z, m):
    n = len(a) # dim of a, z, Q(nxn)
    m = m # Is this to choose or somehow given?
    l = m+1 # length of the array with all a_i in it i:{0,m}
    H = lambda k: optim_tools._H(k, n) # Convinence function to fix n and make specialized H(k)
    N = optim_tools._N(n)

    a_list = [np.zeros((n, n))] * l
    for i in range(0, l): # m eingeschlossen
        if i <= (m-1)/2.0: # 0 <= i <= (m-1)/2
            for k in range(1, i+1):
                a_list[i] += a.T * H(n+k-i)*Q*N*H(n-k+1)*a - z.T*N*H(n-i)*a
        elif i <= m: # (m-1)/2 < i <=m
            for k in range(1, 2*n-i+1):
                a_list[i] += a.T * H(k)*Q*N*H(2*n-i-k+1)*a
        else:
            # this branch is currently not possible in this program
            print "i={} < m={}".format(i, m)
            a_list[i] = 0
            
    return a_list


In [ ]:
from scipy.special import comb as nchoosek # n Choose k (n ueber k)

def trans_a_list(a_list, eps):
    l = len(a_list)
    m = l-1 #biggest index in a_list
    a1_list = [np.zeros(a_list[0].shape)] * l
    for j in range(0, l): #for each in a_list
        for i in range(j, l): # for each coefficient including m
            a1_list[j] += nchoosek(i, i-j) * ((1.0+eps)/(1.0-eps))**(i-j) * ((1.0-eps)/(2.0))**i * a_list[i]
    return a1_list

In [ ]:
def calc_lmi_cond(a_sum, n):
    k = a_sum.size[1] / n # This dimension is from Boris. Dilyana does not specifiy the dimension though

    J = np.hstack([np.zeros((n*(k-1), n)), np.eye(n*(k-1))])
    C = np.hstack([np.eye(n*(k-1)), np.zeros((n*(k-1), n))])
    return np.vstack([C, J]), n*(k-1)

In [ ]:
def calc_a_Sum(a_list):
    l = len(a_list) # number of matrizen in a_list
    m = l-1 # Index of Q_m
    n = a_list[0].size[0] # shape of each matrix, first element
    
    if m is 0:
        a_sum = cvxpy.bmat([[2*a_list[0],   np.zeros(n)], 
                            [np.zeros(n), np.zeros(n)]])
    elif m is 1:
        a_sum = cvxpy.bmat([[2*a_list[0], a_list[1]],
                            [a_list[1],   np.zeros(n)]])
    else: # e.g. m is 2 or more
        a_sum = cvxpy.bmat([[2*a_list[0], a_list[1]],
                            [a_list[1], 2*a_list[2]]])

    for i1 in range(3, l, 2):
        S_new_col = cvxpy.vstack(np.zeros((((i1+1)/2-1)*n, n)), a_list[i1])

        if i1 is m:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), a_list[i1], np.zeros((n,n)))
        else:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), a_list[i1], 2*a_list[i1+1])

        a_sum = cvxpy.bmat([[a_sum, S_new_col],
                            [S_new_row]])

    a_sum = -0.5*a_sum
    
    return a_sum

In [ ]:
a_list = get_a_list(a, Q, z, 4) # Matrizes of Polynom coefficients: S(p)=S_0 + S_1*p + ... S_n*p^n

a1_list = trans_a_list(a_list, 0.01) # Intervaltransformation p:[0,1] -> p1:[-1,1] (S1 -> S^tilde)
a1_sum = calc_a_Sum(a1_list) # S^tilde_sum Matrix (30)
CJ, l = calc_lmi_cond(a1_sum ,n) # "Selection matrizes" of (31), l=dimension of P and G

In [ ]:
##############################
# Convex Problem (4.58)      #
##############################

# Variables
Q  = cvxpy.Variable(n,n) # Implys (459) (semidefinite for numerical reasons?)

z = cvxpy.Variable(n)

# Preparation for constraint (31)
a_list = get_a_list(a, Q, z, 4) # Matrizes of Polynom coefficients: S(p)=S_0 + S_1*p + ... S_n*p^n
a1_list = trans_a_list(a_list, 0.01) # Intervaltransformation p:[0,1] -> p1:[-1,1] (S1 -> S^tilde)
a1_sum = calc_a_Sum(a1_list) # S^tilde_sum Matrix (30)
CJ, l = calc_lmi_cond(a1_sum ,n) # "Selection matrizes" of (31), l=dimension of P and G

# Furhter variables of optimization
S = cvxpy.Variable(l,l) #symmetrical
G = cvxpy.Variable(l,l) #skew

# Help variable to minimize geomean
v = cvxpy.Variable(1)

# Constants
N = optim_tools._N(n)

# Parameters
#beta = cvxpy.Parameter(sign='positive') # Design parameter beta >=1; upper bound of saturation
#beta.value = beta_val


# Constraints
constraint_459 = Q >> 0
constraint_460 = Q*(A.T + a*b.T) + (A + b*a.T)*Q - z*b.T - b*z.T << 0
constraint_461 = Q*N+N*Q << 0

constraint_462 = [cvxpy.bmat([[1,       X0[i].T],
                              [X0[i],    Q      ]]) >> 0
                                for i in range(0, len(X0))]

constraint_463 = cvxpy.bmat([[u_max**2 - a.T*Q*a + 2*a.T*z, z.T],
                             [z,                           Q  ]]) >> 0



constraint_G = G + G.T == 0 # skew symmetry
constraint_S = S == S.T # symmetry

constraint_465 = a1_sum << CJ.T * cvxpy.bmat([[-S,  G],
                                              [G.T, S]]) * CJ

# Objective representation
constraint_obj_451 = v >= -cvxpy.geo_mean(cvxpy.log_det(Q))
#constraint_obj_451 = v >= -cvxpy.geo_mean(Q)

# Collection of all constraints
constraints = [constraint_459]
constraints.append(constraint_460)
constraints.append(constraint_461)
constraints.extend(constraint_462) ##!! Beware of the "extend" if input is array
constraints.append(constraint_463)
constraints.append(constraint_G)
constraints.append(constraint_S)
constraints.append(constraint_465)

#constraints.append(constraint_obj_451)

#obj = cvxpy.Minimize(v)
#obj = cvxpy.Minimize(0)
obj = cvxpy.Minimize(cvxpy.trace(Q))
#obj = cvxpy.Maximize(cvxpy.lambda_min(Q))

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)
prob.solve(solver=cvxpy.SCS, verbose=False, max_iters=150000)
print prob.status
Q.value

Some checks for validy of solution

In [ ]:
import numpy as np
print "Checking constraints (no output is ok!)"
eig_459 = np.linalg.eigvals(Q.value)
if np.any(eig_459 <= 0): print("Q not pos definit: {}".format(eig_459))

eig_460 = np.linalg.eigvals((Q*(A.T+a*b.T)+(A+b*a.T)*Q-z*b.T-b*z.T).value)
if np.any(eig_460 >= 0): print("4.60 not neg definit: {}".format(eig_460))

eig_461 = np.linalg.eigvals((Q*N+N*Q).value)
if np.any(eig_461 >= 0): print("4.61 not neg definit: {}".format(eig_461))
    


Helper functions to run simulation

In [ ]:
# Defining functions
def get_g(p, x, R1):
    try:
        p = p.squeeze() # This is weird
    except:
        pass
    D_inv = _D_inv(p, n)
#    print "p:", p
#    print "D_inv:", D_inv
#    print "x_T:", x.transpose()
#    print "x:", x
#    print "R1:", R1
    g = x.transpose().dot(D_inv).dot(R1).dot(D_inv).dot(x) - 1.0
    # Update 2016: As of python 3.5, there is a new matrix_multiply symbol, @:
    # g = x' @ D^-1 @ R1 @ D^-1 @ x - 1.0
    return g.squeeze()
print "Test g\n", get_g(0.1, np.matrix(range(0,n)).T, R1)

def get_kstar(p, a, a_hat_star):
    try:
        p = p.squeeze() # This is weird
    except:
        pass
    D_inv = _D_inv(p, n)
    kstar = D_inv.dot(a_hat_star) - a
    return kstar
print "Test kstar\n", get_kstar(2, a, a_hat)

def sat(v, u_max, u_min=None):
    if not u_min:
        u_min = -u_max
    return np.clip(v, u_min, u_max)

Simulation of WSVC Control

In [ ]:
from IPython.display import clear_output
from __future__ import division
from scipy.optimize import minimize

#Introduced for fun
u_max_sys = u_max

# Numerical approach
max_T = 120 # Seconds

#simulation time
T = 1.0/100.0

max_iter = int(max_T/T)

# Initial state
x0 = np.array([[0.9],[0]])

# Make functionpointer
func_g = lambda p: np.absolute(get_g(p, x_t, R1))
func_kstar = lambda p: get_kstar(p, a, a_hat)

init_p = 0

p_t = np.zeros(max_iter)
p_t2 = np.zeros(max_iter)
u_t = np.zeros(max_iter)
u_t2 = np.zeros(max_iter)

# Timeloop
x_t = x0
y_t = np.zeros(max_iter)
y_t[0] = x0[0]

with Timer():
    for t in range(1, max_iter):
    #for t in range(0, 10000):
        if t%1000 == 1:
            clear_output()
            print t*T, "seconds done ->", t/max_iter*100, "%"
        ## Calc p
        res = minimize(func_g, p_t2[t-1], method='Nelder-Mead')
        #print res.x

        #g_poly = sp.Poly(sys, p)
        #show(g_poly)
        p = sat(res.x, 1, 0.04)
        p_t[t] = res.x
        p_t2[t] = p

        ## Calc u
        k_t = func_kstar(p)
        u = np.dot(np.array(-k_t.T), x_t)
        #print u_sym
        u_t[t] = u
        u = sat(u, u_max)
        u_t2[t] = u

        ## System response
        # System saturation u (trivial if u_max and u_max_sys are identical)
        # Calc x_dot
        x_dot = np.dot(A, x_t) + b * sat(u, u_max_sys)

        x_t = x_t + x_dot*T
        y_t[t] = x_t[0]
    clear_output()

print "done"

Additional simulation of simple SS Control

In [ ]:
ux_t = np.zeros(max_iter)
ux_t2 = np.zeros(max_iter)
yx_t = np.zeros(max_iter)

# Timeloop
x_t = x0
yx_t[0] = x0[0]

with Timer():
    for t in range(1, max_iter):
    #for t in range(0, 10000):
        if t%1000 == 1:
            clear_output()
            print t*T, "seconds done ->", t/max_iter*100, "%"
        
        ## Calc u
        k_t = func_kstar(0.5)
        u = np.dot(np.array(-k_t.T), x_t)
        #print u_sym
        ux_t[t] = u
        u = sat(u, u_max)
        ux_t2[t] = u

        ## System response
        # System saturation u (trivial if u_max and u_max_sys are identical)
        # Calc x_dot
        x_dot = np.dot(A, x_t) + b * sat(u, u_max_sys)

        x_t = x_t + x_dot*T
        yx_t[t] = x_t[0]
    clear_output()

print "done"

Plotting

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 20)

plt.subplot(311)
plt.plot(np.arange(0, len(y_t))*T, y_t, 'b')
plt.plot(np.arange(0, len(yx_t))*T, yx_t, 'b.')
plt.xlabel('t [s]')
plt.ylabel('y(t)')

plt.subplot(312)
plt.plot(np.arange(0, len(y_t))*T, p_t, 'g--')
plt.plot(np.arange(0, len(y_t))*T, p_t2, 'g')
plt.xlabel('t [s]')
plt.ylabel('p(t)')

plt.subplot(313)
axes = plt.gca()
axes.set_ylim([-u_max-1e-5,u_max+1e-5])

plt.plot(np.arange(0, len(y_t))*T, u_t, 'r--')
plt.plot(np.arange(0, len(y_t))*T, u_t2, 'r')
plt.plot(np.arange(0, len(y_t))*T, ux_t, 'y--')
plt.plot(np.arange(0, len(y_t))*T, ux_t2, 'y')
plt.xlabel('t [s]')
plt.ylabel('u(t)')

plt.show()

In [ ]:
print np.array(eps.value) > 0


(Yet failing) visualtions of Ljaponov regions

In [ ]:
# V(x) = x.T * P * x
def V(x,y,P):
    x = np.matrix([x, y]).T
    return x.T * P * x

arr = [(x,y,V(x,y,R1)) for x in np.arange(-1, 1, 0.1) for y in np.arange(-1, 1, 0.1)]

import matplotlib.pyplot as plt
plt.plot(arr)
plt.show()
print arr